# Course1-Project1: EdTech Backend System

In [64]:
import hashlib
from openpyxl import load_workbook
#from enum import Enum
#class Role(Enum):
#    LEARNER = "Learner"
#    INSTRUCTOR = "Instructor"

file_path = r"C:\Users\ecarald\OneDrive - Ericsson\Desktop\PG DE\Course 1\instructors.xlsx"

def set_id(name):
    # Encode the string and compute a hash (SHA-256)
    hash_name = hashlib.sha256(name.encode('utf-8'))
    # Convert the hexadecimal hash to an integer
    id = int(hash_name.hexdigest(), 16)
    # Take only part of it (last 10 digits)
    return id % (10**10)
        
class User:
    counter = 0
    
    def __init__(self, name, role=None, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'learner' or 'instructor'
        'email' and 'password' are optional.
        '''
        #if not isinstance(role,Role):
        #    raise ValueError("role must be a member of Role Enum.")
            
        self.name = name
        self.id=set_id(name)
        self.role = role
        self.email = email
 
        if password:
            self.hash_password(password)
            self.dotify_password(password)

        User.counter +=1
        
    def set_email(self, email):
        self.email=email

    def hash_password(self, password):
        '''
        Hash the password using SHA-256 before storing it
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed
        
    def dotify_password(self, password):
        '''
        Mask the password with dots
        '''
        self.dotified_password = '.' * len(password)

    def validate_password(self,password):
        """
        Check if a given password matches the stored hashed password.
        """
        if self.__password_hash is None:
            return False
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        return hashed == self.__password_hash
        
    def validate_email(self,email):
        return self.email == email            

    def set_role(self):
        option = int(input("Choose either (1) for 'Learner' or (2) for 'Instructor': "))
        if option == 1:
            self.role = "Learner"
        elif option == 2:
            self.role = "Instructor"
        else:
            raise ValueError("Wrong value. Role can only be either (1) for 'Learner' or (2) for 'Instructor")
        return self.role
        
    def print_info(self):
        print("Name    :", self.name)
        print("User Id :", self.id)
        print("Role    :", self.role)
        print("Email   :", self.email)
        print("Password:", self.dotified_password)


class Instructor(User):
    def __init__(self, name, email, taught_courses):
        User.__init__(self, name, "Instructor", email)
        #self.taught_courses = taught_courses # 'taught_courses' is a simple list of all the courses that instructor teaches
        self.taught_courses = taught_courses if taught_courses else [] #elegante

    def add_course(self, course_name):
        '''Adds a course to the instructor's taught_courses list if not already present.'''
        if course_name not in self.taught_courses:
            self.taught_courses.append(course_name)
            print(f"Course '{course_name}' added to {self.name}.")
        else:
            print(f"{self.name} already teaches '{course_name}'.")

    def remove_course(self, course_name):
        '''Removes course from the instructor´s taught_courses list if it exists.'''
        if course_name in self.taught_courses:
            self.taught_courses.remove(course_name)
            print(f"Course {course_name} removed from {self.name}.")
        else:
            print(f"{self.name} does not teaches '{course_name}'.")
    
    def __str__(self):
        return f"Instructor name: {self.name}\nInstructor ID: {self.id}\nTaught courses: {self.taught_courses}"


class Course:
    def __init__(self, name, instructors):
        '''Build course from instructors list'''
        self.name = name
        self.id = set_id(name)
        self.instructors = instructors

    def __str__(self):
        # Convert instructor objects into "{name : id}" format
        instructors_str = ", ".join(
            [f"{{{inst.name} : {inst.id}}}" for inst in self.instructors]
        )
        return f"Course name: {self.name}\nCourse ID: {self.id}\nInstructors: {instructors_str}"
        

class Backend():
    def __init__(self): 
        self.instructors = [] #'instructors' is a simple list of instances of class Instructor

    def load_instructors_from_excel(self, file_path):           
        # Load workbook and select first sheet
        workbook = load_workbook(filename=file_path)
        sheet = workbook.active        

        # Skip the header row → start from row 2
        for row in sheet.iter_rows(min_row=2, values_only=True):
            name = row[0]
            surname = row[1]
            email = row[2]
            # OBS! courses is a simple list of course names (!= instances of class Course)
            courses = [cell for cell in row[3:] if cell is not None]  # skip empty cells

            full_name = f"{name} {surname}"
            instructor = Instructor(full_name, email, courses)
            self.instructors.append(instructor)

    def generate_courses_from_instructors(self):
        '''
        Creates Course objects based on instructors' taught courses.
        Each Course has a name and a list of instructor names teaching it.
        '''
        courses_dict = {} #´course_name -> list of instructor names

        #Build mapping from course -> instructor list
        for instructor in self.instructors:
            for course_name in instructor.taught_courses:
                if course_name not in courses_dict:
                    courses_dict[course_name] = [] # initialize new course entry with key ´course-name'
                courses_dict[course_name].append(instructor) # append the full instructor object

        # Convert mapping into Course objects
        self.courses = [
            Course(name, instructors_list)
            for name, instructors_list in courses_dict.items()
        ]
    
    def add_instructor(self, new_instructor):
        '''Add a new instructor to the list if the ID does not already exist.'''
        #Check if instructor.id already exists
        existing = next((i for i in self.instructors if i.id == new_instructor.id), None)
        
        if existing:
            print(f"Instructor with ID {new_instructor.id} already exists: {existing.name}")
        else:    
            self.instructors.append(new_instructor)
            print(f"Instructor '{new_instructor.name}' added successfully.")
    
    def add_course_to_instructor(self, instructor_id, course_name):
        '''Find instructor by ID, add course, and update global course list.'''
        instructor = next((i for i in self.instructors if i.id == instructor_id), None)

        if not instructor:
            print(f"No instructor found with ID {instructor_id}.")
            return

        # Step 1: Add course to instructor’s list
        instructor.add_course(course_name)

        # Step 2: Check if course already exists globally
        existing_course = next((c for c in self.courses if c.name == course_name), None)
        if existing_course:
            # If the course exists, add this instructor if not already there
            if instructor not in existing_course.instructors:
                existing_course.instructors.append(instructor)
        else:
            # Otherwise, create a new Course instance
            new_course = Course(course_name, [instructor])
            self.courses.append(new_course)
            print(f"New course '{course_name}' created.")
        
    def remove_course_from_instructor(self, instructor_id, course_name):
        '''Find instructor by ID, remove course, and update global course list'''
        instructor = next((i for i in self.instructors if i.id == instructor_id), None)

        if instructor is None:
            print(f"No instructor found with ID {instructor_id}.")
            return

        # Step 1: Remove course from instructor’s list
        instructor.remove_course(course_name)

        # Step 2: Remove instructor from global instructors' list if that course was the only one instructor taught
        if not instructor.taught_courses: # Check if taught_courses list is empty
            self.instructors.remove(instructor)

        # Step 3: Check if course already exisits globally
        existing_course = next((c for c in self.courses if c.name == course_name), None)
        if existing_course:
            # If the course exists and there is only one instructor associated with it, then remove course
            if len(existing_course.instructors) == 1:
                self.courses.remove(course_name)
        else:
            print(f"No course found for instructor ID {instructor_id}.")
            
            
    def _print_entity_info(self, name, data_list, entity_label):
        '''Private helper to print an entity from a list, given the name.''' #todo: It might be interesting to also have the possibility to search by ID
        found = next((obj for obj in data_list if obj.name == name), None)
        if found:
            print(found)
        else:
            print(f"{entity_label} '{name}' not found.")

    def print_course_info(self, course_name):
        self._print_entity_info(course_name, self.courses, "Course")

    def print_instructor_info(self, instructor_name):
        self._print_entity_info(instructor_name, self.instructors, "Instructor")    
    

############################################################################################
backend = Backend()
backend.load_instructors_from_excel(file_path)
print("List of all available instructors:")
for instructor in backend.instructors:
    print(instructor)
    print("-"*30)

backend.generate_courses_from_instructors()
print("\nList of all available courses:")
for course in backend.courses:
    print(course)
    print("-"*30)

# Testing of add_course_to_instructor method
i1 = Instructor("Carlos Ccc", "cc@mail.com", ["Algebra", "Biology"])
i2 = Instructor("Daniel Ddd", "dd@mail.com", ["Biology"])
i3 = Instructor("Alice Aaa", "aa@mail.com", ["Dentist"])
i4 = Instructor("Eric Eee", "ee@mail.com", ["Pharmacy"])
backend.add_instructor(i1)
#backend.add_instructor(i2)
#backend.add_instructor(i3) # Note that i3.mail already exists!
#backend.add_instructor(i4)

backend.add_course_to_instructor(i1.id,"Calculus")
backend.print_instructor_info(i1.name)

#backend.add_course_to_instructor(i2.id,"Biology") # Note that already exists

#backend.remove_course_from_instructor(i2.id,"Biology")
#backend.print_course_info("Biology")

#backend.remove_course_from_instructor(i4.id,"Pharmacy")
#backend.print_course_info("Pharmacy") # Note course 'Pharmacy' no longer exist in courses global list




List of all available instructors:
Instructor name: Alice Aaa
Instructor ID: 5535300755
Taught courses: ['Algebra', 'Biology', 'Calculus']
------------------------------
Instructor name: Bob Bbb
Instructor ID: 6623670666
Taught courses: ['Algebra', 'Calculus', 'Dentist']
------------------------------

List of all available courses:
Course name: Algebra
Course ID: 2872948788
Instructors: {Alice Aaa : 5535300755}, {Bob Bbb : 6623670666}
------------------------------
Course name: Biology
Course ID: 9230804429
Instructors: {Alice Aaa : 5535300755}
------------------------------
Course name: Calculus
Course ID: 3793446076
Instructors: {Alice Aaa : 5535300755}, {Bob Bbb : 6623670666}
------------------------------
Course name: Dentist
Course ID: 9540905890
Instructors: {Bob Bbb : 6623670666}
------------------------------
Instructor 'Carlos Ccc' added successfully.
Course 'Calculus' added to Carlos Ccc.
Instructor name: Carlos Ccc
Instructor ID: 1033751262
Taught courses: ['Algebra', 'Biol